In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [2]:
import os
if not os.path.isdir('saved_models_{}'.format('lsgan')):
    os.mkdir('saved_models_{}'.format('lsgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('lsgan'), mode='w')
import cv2
import torch.utils.data as Data

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class LSGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(z, valid)
        # (!!!) Optimize w.r.t. MSE loss instead of crossentropy
        self.combined.compile(loss='mse', optimizer=optimizer)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_test), (X_test, _) = fashion_mnist.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        X_test = np.expand_dims(X_test, axis=3)
        # y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, global_step,d_loss[0], 100 * d_loss[1], g_loss))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')



if __name__ == '__main__':
    gan = LSGAN()
    gan.train(epochs=30, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.296361, acc.: 42.19%] [G loss: 0.271252]
epoch:0 step:2 [D loss: 0.248585, acc.: 55.47%] [G loss: 0.316814]
epoch:0 step:3 [D loss: 0.230560, acc.: 66.41%] [G loss: 0.323316]
epoch:0 step:4 [D loss: 0.204065, acc.: 67.97%] [G loss: 0.334172]
epoch:0 step:5 [D loss: 0.169514, acc.: 75.78%] [G loss: 0.342239]
epoch:0 step:6 [D loss: 0.172839, acc.: 73.44%] [G loss: 0.345749]
epoch:0 step:7 [D loss: 0.152901, acc.: 77.34%] [G loss: 0.370201]
epoch:0 step:8 [D loss: 0.166923, acc.: 78.12%] [G loss: 0.415393]
epoch:0 step:9 [D loss: 0.142284, acc.: 85.16%] [G loss: 0.448127]
epoch:0 step:10 [D loss: 0.143492, acc.: 79.69%] [G loss: 0.441695]
epoch:0 step:11 [D loss: 0.143367, acc.: 81.25%] [G loss: 0.425767]
epoch:0 step:12 [D loss: 0.132634, acc.: 83.59%] [G loss: 0.402843]
epoch:0 step:13 [D loss: 0.145742, acc.: 78.12%] [G loss: 0.438112]
epoch:0 step:14 [D loss: 0.151357, acc.: 75.00%] [G loss: 0.415675]
epoch:0 step:15 [D loss: 0.134294, acc.: 78.91%] [G loss:

epoch:0 step:124 [D loss: 0.209008, acc.: 65.62%] [G loss: 0.547885]
epoch:0 step:125 [D loss: 0.198438, acc.: 75.78%] [G loss: 0.567179]
epoch:0 step:126 [D loss: 0.185471, acc.: 70.31%] [G loss: 0.564152]
epoch:0 step:127 [D loss: 0.183952, acc.: 71.88%] [G loss: 0.557986]
epoch:0 step:128 [D loss: 0.191882, acc.: 71.09%] [G loss: 0.554040]
epoch:0 step:129 [D loss: 0.182543, acc.: 74.22%] [G loss: 0.535708]
epoch:0 step:130 [D loss: 0.214142, acc.: 69.53%] [G loss: 0.518125]
epoch:0 step:131 [D loss: 0.221590, acc.: 64.06%] [G loss: 0.528299]
epoch:0 step:132 [D loss: 0.162572, acc.: 76.56%] [G loss: 0.600311]
epoch:0 step:133 [D loss: 0.201900, acc.: 67.19%] [G loss: 0.525368]
epoch:0 step:134 [D loss: 0.185692, acc.: 71.09%] [G loss: 0.482889]
epoch:0 step:135 [D loss: 0.216458, acc.: 64.84%] [G loss: 0.453846]
epoch:0 step:136 [D loss: 0.220759, acc.: 65.62%] [G loss: 0.513325]
epoch:0 step:137 [D loss: 0.193662, acc.: 70.31%] [G loss: 0.512376]
epoch:0 step:138 [D loss: 0.225029

...

epoch:29 step:27576 [D loss: 0.247932, acc.: 58.59%] [G loss: 0.288521]
epoch:29 step:27577 [D loss: 0.237418, acc.: 61.72%] [G loss: 0.285572]
epoch:29 step:27578 [D loss: 0.239354, acc.: 64.84%] [G loss: 0.322650]
epoch:29 step:27579 [D loss: 0.235560, acc.: 60.94%] [G loss: 0.294806]
epoch:29 step:27580 [D loss: 0.247047, acc.: 50.00%] [G loss: 0.314273]
epoch:29 step:27581 [D loss: 0.237877, acc.: 55.47%] [G loss: 0.287501]
epoch:29 step:27582 [D loss: 0.233884, acc.: 60.16%] [G loss: 0.295001]
epoch:29 step:27583 [D loss: 0.234095, acc.: 56.25%] [G loss: 0.298502]
epoch:29 step:27584 [D loss: 0.253156, acc.: 53.91%] [G loss: 0.308563]
epoch:29 step:27585 [D loss: 0.233103, acc.: 63.28%] [G loss: 0.276028]
epoch:29 step:27586 [D loss: 0.242825, acc.: 56.25%] [G loss: 0.291620]
epoch:29 step:27587 [D loss: 0.249211, acc.: 60.16%] [G loss: 0.277988]
epoch:29 step:27588 [D loss: 0.233997, acc.: 54.69%] [G loss: 0.296317]
epoch:29 step:27589 [D loss: 0.259980, acc.: 54.69%] [G loss: 0.

epoch:29 step:27692 [D loss: 0.225577, acc.: 55.47%] [G loss: 0.302206]
epoch:29 step:27693 [D loss: 0.231836, acc.: 62.50%] [G loss: 0.286819]
epoch:29 step:27694 [D loss: 0.227667, acc.: 58.59%] [G loss: 0.314639]
epoch:29 step:27695 [D loss: 0.246201, acc.: 53.12%] [G loss: 0.292040]
epoch:29 step:27696 [D loss: 0.228728, acc.: 62.50%] [G loss: 0.295121]
epoch:29 step:27697 [D loss: 0.225180, acc.: 62.50%] [G loss: 0.322611]
epoch:29 step:27698 [D loss: 0.253229, acc.: 57.03%] [G loss: 0.293146]
epoch:29 step:27699 [D loss: 0.239926, acc.: 60.16%] [G loss: 0.307517]
epoch:29 step:27700 [D loss: 0.240163, acc.: 55.47%] [G loss: 0.335573]
epoch:29 step:27701 [D loss: 0.242097, acc.: 59.38%] [G loss: 0.299779]
epoch:29 step:27702 [D loss: 0.238779, acc.: 57.81%] [G loss: 0.284953]
epoch:29 step:27703 [D loss: 0.244939, acc.: 56.25%] [G loss: 0.274772]
epoch:29 step:27704 [D loss: 0.247266, acc.: 53.12%] [G loss: 0.304534]
epoch:29 step:27705 [D loss: 0.235054, acc.: 62.50%] [G loss: 0.

epoch:29 step:27805 [D loss: 0.248651, acc.: 55.47%] [G loss: 0.297805]
epoch:29 step:27806 [D loss: 0.243335, acc.: 57.81%] [G loss: 0.269648]
epoch:29 step:27807 [D loss: 0.240552, acc.: 58.59%] [G loss: 0.269150]
epoch:29 step:27808 [D loss: 0.229500, acc.: 61.72%] [G loss: 0.286105]
epoch:29 step:27809 [D loss: 0.243066, acc.: 58.59%] [G loss: 0.299568]
epoch:29 step:27810 [D loss: 0.254056, acc.: 53.91%] [G loss: 0.285378]
epoch:29 step:27811 [D loss: 0.236583, acc.: 59.38%] [G loss: 0.281039]
epoch:29 step:27812 [D loss: 0.247223, acc.: 55.47%] [G loss: 0.295657]
epoch:29 step:27813 [D loss: 0.237110, acc.: 60.94%] [G loss: 0.327321]
epoch:29 step:27814 [D loss: 0.242373, acc.: 57.03%] [G loss: 0.273788]
epoch:29 step:27815 [D loss: 0.229613, acc.: 58.59%] [G loss: 0.303037]
epoch:29 step:27816 [D loss: 0.256311, acc.: 53.12%] [G loss: 0.277550]
epoch:29 step:27817 [D loss: 0.233487, acc.: 58.59%] [G loss: 0.287266]
epoch:29 step:27818 [D loss: 0.223892, acc.: 61.72%] [G loss: 0.

epoch:29 step:27923 [D loss: 0.238464, acc.: 59.38%] [G loss: 0.322541]
epoch:29 step:27924 [D loss: 0.232676, acc.: 64.06%] [G loss: 0.286312]
epoch:29 step:27925 [D loss: 0.220329, acc.: 62.50%] [G loss: 0.313505]
epoch:29 step:27926 [D loss: 0.234094, acc.: 60.16%] [G loss: 0.301221]
epoch:29 step:27927 [D loss: 0.238273, acc.: 58.59%] [G loss: 0.316097]
epoch:29 step:27928 [D loss: 0.234611, acc.: 64.06%] [G loss: 0.257009]
epoch:29 step:27929 [D loss: 0.242525, acc.: 57.81%] [G loss: 0.300021]
epoch:29 step:27930 [D loss: 0.233696, acc.: 57.03%] [G loss: 0.303531]
epoch:29 step:27931 [D loss: 0.234302, acc.: 60.16%] [G loss: 0.293804]
epoch:29 step:27932 [D loss: 0.238270, acc.: 59.38%] [G loss: 0.275828]
epoch:29 step:27933 [D loss: 0.236983, acc.: 57.03%] [G loss: 0.300443]
epoch:29 step:27934 [D loss: 0.239144, acc.: 61.72%] [G loss: 0.312643]
epoch:29 step:27935 [D loss: 0.267307, acc.: 45.31%] [G loss: 0.258661]
epoch:29 step:27936 [D loss: 0.249608, acc.: 53.91%] [G loss: 0.

epoch:29 step:28035 [D loss: 0.234262, acc.: 58.59%] [G loss: 0.286625]
epoch:29 step:28036 [D loss: 0.219454, acc.: 68.75%] [G loss: 0.312307]
epoch:29 step:28037 [D loss: 0.236791, acc.: 62.50%] [G loss: 0.290281]
epoch:29 step:28038 [D loss: 0.242474, acc.: 54.69%] [G loss: 0.314985]
epoch:29 step:28039 [D loss: 0.244511, acc.: 55.47%] [G loss: 0.284969]
epoch:29 step:28040 [D loss: 0.239492, acc.: 58.59%] [G loss: 0.266904]
epoch:29 step:28041 [D loss: 0.230552, acc.: 58.59%] [G loss: 0.320472]
epoch:29 step:28042 [D loss: 0.223606, acc.: 65.62%] [G loss: 0.299991]
epoch:29 step:28043 [D loss: 0.249658, acc.: 50.78%] [G loss: 0.305976]
epoch:29 step:28044 [D loss: 0.233652, acc.: 60.94%] [G loss: 0.312880]
epoch:29 step:28045 [D loss: 0.253048, acc.: 46.88%] [G loss: 0.292442]
epoch:29 step:28046 [D loss: 0.229863, acc.: 61.72%] [G loss: 0.299232]
epoch:29 step:28047 [D loss: 0.240836, acc.: 60.16%] [G loss: 0.285461]
epoch:29 step:28048 [D loss: 0.233999, acc.: 60.16%] [G loss: 0.